# Prompt Templates

Um prompt para um modelo de linguagem é um conjunto de instruções ou entradas fornecidas por um usuário para guiar a resposta do modelo, ajudando-o a entender o contexto e gerar uma saída baseada em linguagem relevante e coerente, como responder a perguntas, completar frases ou participar de uma conversa.

In [1]:
!pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.7
    Uninstalling langchain-core-0.3.7:
      Successfully uninstalled langchain-core-0.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.1 requires langchain-core<0.4.0,>=0.3.6, but you have langchain-core 0.1.52 which is incompatible.
langchain-text-splitters 0.3.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.1.52 which is incompatible.
langchain-community 0.3.1 requires langchain-core<0.4.0,>=0.3.6, but you have langchain-core 0.1.52 which is incompatible.


In [1]:
from langchain_openai.llms import OpenAI

llm = OpenAI()

In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário:
{pergunta}                                   
''')
prompt_template

PromptTemplate(input_variables=['pergunta'], template='\nResponda a seguinte pergunta do usuário:\n{pergunta}                                      \n')

In [4]:
print(prompt_template.format(pergunta='O que é um buraco negro?'))


Responda a seguinte pergunta do usuário:
O que é um buraco negro?                                      



In [6]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palvras} palavras:
{pergunta}                                   
''')
prompt_template.format(n_palvras=10, pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 10 palavras:\nO que é um buraco negro?                                   \n'

In [8]:
prompt_template = PromptTemplate.from_template('''
Responda a seguinte pergunta do usuário em até {n_palvras} palavras:
{pergunta}                                   
''', partial_variables={'n_palvras': 10})# default 10
prompt_template.format(pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 10 palavras:\nO que é um buraco negro?                                   \n'

In [9]:
prompt_template.format(n_palvras=5, pergunta='O que é um buraco negro?')

'\nResponda a seguinte pergunta do usuário em até 5 palavras:\nO que é um buraco negro?                                   \n'

## Composing prompts | Unindo múltiplos prompts

In [16]:
from langchain.prompts import PromptTemplate

template_word_count = PromptTemplate.from_template('''
Responda a pergunta em até {n_palavras} palavras.''')

template_lingua = PromptTemplate.from_template('''
Retorne a resposta na {lingua}.''')

template_final = (
    template_word_count
    + template_lingua
    + '\nResponda a pergunta seguinte seguindo as instruções: {pergunta}'
)

template_final.template

'\nResponda a pergunta em até {n_palavras} palavras.\nRetorn a resposta na {lingua}.\nResponda a pergunta seguinte seguindo as instruções: {pergunta}'

In [ ]:
prompt = template_final.format(n_palavras=10, lingua='inglês', pergunta='O que é uma estrela?')
print(prompt)


Responda a pergunta em até 10 palavras.
Retorn a resposta na inglês.
Responda a pergunta seguinte seguindo as instruções: O que é uma estrela?


In [14]:
prompt = template_final.format(n_palavras=10, lingua='inglês', pergunta='O que é uma estrela?')
llm.invoke(prompt)

'\n\nA star is a luminous ball of gas.'

## Templates para Chat

In [19]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_template('Essa é a minha dúvida: {duvida}')
chat_template.format_messages(duvida='Quem sou eu?')

[HumanMessage(content='Essa é a minha dúvida: Quem sou eu?')]

In [20]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'Você é um assistente engraçado e se chama {nome_assistente}'),
        ('human', 'Olá, como vai?'),
        ('ai', 'Melhor agora! como posso ajudá-lo?'),
        ('human', '{pergunta}')
    ]
)

chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente engraçado e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá, como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Melhor agora! como posso ajudá-lo?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [21]:
chat_template.format_messages(nome_assistente='Asimo', pergunta='Qual o seu nome?')

[SystemMessage(content='Você é um assistente engraçado e se chama Asimo'),
 HumanMessage(content='Olá, como vai?'),
 AIMessage(content='Melhor agora! como posso ajudá-lo?'),
 HumanMessage(content='Qual o seu nome?')]

In [22]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()

chat.invoke(chat_template.format_messages(nome_assistente='Asimo', pergunta='Qual o seu nome?'))

AIMessage(content='Meu nome é Asimo, o assistente engraçado! Como posso alegrar o seu dia hoje?', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 56, 'total_tokens': 82}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-1ac33b1c-773a-4e73-aeec-dc8c4c535763-0')

## Templates de Few-shot prompting para llm

In [23]:
exemplos = [
    {"pergunta": "Quem viveu mais tempo, Muhammad Ali ou Alan Turing?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 
""", 
    }, 
    {"pergunta": "Quando nasceu o fundador do craigslist?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 
""", 
    }, 
    {"pergunta": "Quem foi o avô materno de George Washington?",
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi a mãe de George Washington? 
Resposta intermediária: A mãe de George Washington foi Mary Ball Washington. 
Pergunta de acompanhamento: Quem foi o pai de Mary Ball Washington? 
Resposta intermediária: O pai de Mary Ball Washington foi Joseph Ball. 
Então a resposta final é: Joseph Ball 
""", 
    },
    {"pergunta": "Os diretores de Jaws e Casino Royale são do mesmo país?", 
     "resposta": 
"""São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem é o diretor de Jaws? 
Resposta Intermediária: O diretor de Jaws é Steven Spielberg. 
Pergunta de acompanhamento: De onde é Steven Spielberg? 
Resposta Intermediária: Estados Unidos. 
Pergunta de acompanhamento: Quem é o diretor de Casino Royale? 
Resposta Intermediária: O diretor de Casino Royale é Martin Campbell. 
Pergunta de acompanhamento: De onde é Martin Campbell? 
Resposta Intermediária: Nova Zelândia. 
Então a resposta final é: Não 
""",
    },
]

In [26]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['pergunta', 'resposta'],
    template='Pergunta {pergunta}\n{resposta}'
)

example_prompt.format(**exemplos[0])


'Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'

In [29]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix='Pergunta: {input}',
    input_variables=['input']
)
prompt

FewShotPromptTemplate(input_variables=['input'], examples=[{'pergunta': 'Quem viveu mais tempo, Muhammad Ali ou Alan Turing?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'}, {'pergunta': 'Quando nasceu o fundador do craigslist?', 'resposta': 'São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. \nEntão a resposta final é: 6 de dezembro de 1952 \n'}, {'pergunta': 'Quem foi 

In [31]:
print(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

Pergunta Quem viveu mais tempo, Muhammad Ali ou Alan Turing?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? 
Resposta intermediária: Muhammad Ali tinha 74 anos quando morreu. 
Pergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? 
Resposta intermediária: Alan Turing tinha 41 anos quando morreu. 
Então a resposta final é: Muhammad Ali 


Pergunta Quando nasceu o fundador do craigslist?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi o fundador do craigslist? 
Resposta intermediária: O craigslist foi fundado por Craig Newmark. 
Pergunta de acompanhamento: Quando nasceu Craig Newmark? 
Resposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. 
Então a resposta final é: 6 de dezembro de 1952 


Pergunta Quem foi o avô materno de George Washington?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: 

In [32]:
llm.invoke(prompt.format(input='Quem fez mais gols, Romário ou Pelé?'))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos gols Romário fez em sua carreira? \nResposta intermediária: Romário fez 772 gols em sua carreira. \nPergunta de acompanhamento: Quantos gols Pelé fez em sua carreira? \nResposta intermediária: Pelé fez 1281 gols em sua carreira. \nEntão a resposta final é: Pelé'

## Templates de Few-shot prompting para chat

In [33]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI()



In [34]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{pergunta}'),
     ('ai', '{resposta}')]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?'), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n')]


In [42]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)
# few_shot_template.format_messages()
prompt_template = ChatPromptTemplate.from_messages(
    [
    few_shot_template,
    ('human', '{input}')
    ]
)

prompt = prompt_template.format_messages(input='Quem fez mais gols, Romário ou Pelé?')
prompt

[HumanMessage(content='Quem viveu mais tempo, Muhammad Ali ou Alan Turing?'),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantos anos Muhammad Ali tinha quando morreu? \nResposta intermediária: Muhammad Ali tinha 74 anos quando morreu. \nPergunta de acompanhamento: Quantos anos Alan Turing tinha quando morreu? \nResposta intermediária: Alan Turing tinha 41 anos quando morreu. \nEntão a resposta final é: Muhammad Ali \n'),
 HumanMessage(content='Quando nasceu o fundador do craigslist?'),
 AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quem foi o fundador do craigslist? \nResposta intermediária: O craigslist foi fundado por Craig Newmark. \nPergunta de acompanhamento: Quando nasceu Craig Newmark? \nResposta intermediária: Craig Newmark nasceu em 6 de dezembro de 1952. \nEntão a resposta final é: 6 de dezembro de 1952 \n'),
 HumanMessage(content='Quem foi o avô materno de

In [43]:
chat.invoke(prompt)

AIMessage(content='O jogador que marcou mais gols em sua carreira foi Pelé, com mais de 1.000 gols registrados. Romário, por sua vez, marcou mais de 700 gols em sua carreira. Portanto, Pelé fez mais gols do que Romário.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 538, 'total_tokens': 601}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-3dd9297b-fdf0-4dbd-9326-9cb019ee3bf6-0')